In [1]:
from tqdm import tqdm
# Alex can I have $50 in AWS Bedrock credits to test some LLM stuff

In [2]:
import pandas as pd

search_space = pd.read_excel('../20240424_trial_searchspace.xlsx').reset_index(drop=True).drop('Unnamed: 0', axis=1)


In [3]:
import outlines

model = outlines.models.transformers("Open-Orca/Mistral-7B-OpenOrca")
# TODO: instruct vs OpenOrca

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
@outlines.prompt #TODO: add value to description (break out point 1) , also check credentials of LLM, think rolling stone example include 
def identify_outcomes(clinical_trial):
    """You are a professional medical practicioner with a medical degree. Other doctors \ 
    send you clinical trial reports from which you need to extract:

    1. The primary outcome metrics from the study
    2. The value measured for each primary outcome
    3. The therapeutic treatment regiment used to achieve each primary outcome
    
    # EXAMPLE
    CLINICAL_TRIAL: The median overall survival was 9.0 months in the triplet-therapy group and 5.4 months in the control group (hazard ratio for death, 0.52; 95% confidence interval [CI], 0.39 to 0.70; P<0.001). The confirmed response rate was 26% (95% CI, 18 to 35) in the triplet-therapy group and 2% (95% CI, 0 to 7) in the control group (P<0.001). The median overall survival in the doublet-therapy group was 8.4 months (hazard ratio for death vs. control, 0.60; 95% CI, 0.45 to 0.79; P<0.001). Adverse events of grade 3 or higher occurred in 58% of patients in the triplet-therapy group, in 50% in the doublet-therapy group, and in 61% in the control group.
    RESULTS: {"outcome": "median overall survival", "value": "9.0 months", "regiment":"triplet-therapy group"}, {"outcome": "median overall survival", "value": "5.4 months", "regiment": "control group"}, {"outcome": "confirmed response rate", "value": "26% (95 CI, 18 to 35)", "regiment": "triplet-therapy-group"}, {"outcome": "confirmed response rate", "value": "2% (95% CI, 0 to 7)", "regiment": "control group"}

    # OUTPUT INSTRUCTIONS    
    
    Answer in valid JSON. Here are different objects relevant for the output:

    ClinicalOutcome:
        outcome (str): name of the primary outcome measure
        value (str): the value that was measured from the outcome
        regiment (str): the therapeutic treatment strategy used to obtain the outcome

    ClinicalOutcomeList:
        outcomes (list): a list of ClinicalOutcomes

    # OUTPUT
    
    CLINICAL_TRIAL: {{ clinical_trial }}
    RESULTS: """

In [5]:
from pydantic import BaseModel

class ClinicalOutcome(BaseModel):
    outcome: str 
    value: str
    regiment: str

class ClinicalOutcomeList(BaseModel):
    outcomes: list[ClinicalOutcome]

In [6]:
prompts = [identify_outcomes(trial) for trial in list(search_space['clinical_studies'][0:31])]
prompts[0:2]

['You are a professional medical practicioner with a medical degree. Other doctors \\ \nsend you clinical trial reports from which you need to extract:\n\n1. The primary outcome metrics from the study\n2. The value measured for each primary outcome\n3. The therapeutic treatment regiment used to achieve each primary outcome\n\n# EXAMPLE\nCLINICAL_TRIAL: The median overall survival was 9.0 months in the triplet-therapy group and 5.4 months in the control group (hazard ratio for death, 0.52; 95% confidence interval [CI], 0.39 to 0.70; P<0.001). The confirmed response rate was 26% (95% CI, 18 to 35) in the triplet-therapy group and 2% (95% CI, 0 to 7) in the control group (P<0.001). The median overall survival in the doublet-therapy group was 8.4 months (hazard ratio for death vs. control, 0.60; 95% CI, 0.45 to 0.79; P<0.001). Adverse events of grade 3 or higher occurred in 58% of patients in the triplet-therapy group, in 50% in the doublet-therapy group, and in 61% in the control group.\n

In [7]:
generator = outlines.generate.json(model, ClinicalOutcomeList)

Compiling FSM index for all state transitions: 100%|██████████| 90/90 [00:00<00:00, 104.95it/s]


In [9]:
results[0]

ClinicalOutcomeList(outcomes=[ClinicalOutcome(outcome='time-weighted summed pain intensity difference over 12 hours (SPID12)', value='time', regiment='DSUVIA 30 mcg'), ClinicalOutcome(outcome='time-weighted summed pain intensity difference over 12 hours (SPID12)', value='placebo', regiment='placebo'), ClinicalOutcome(outcome='median time to onset of meaningful pain relief', value='54 minutes', regiment='DSUVIA group'), ClinicalOutcome(outcome='median time to onset of meaningful pain relief', value='84 minutes', regiment='placebo'), ClinicalOutcome(outcome='percentage of patients taking rescue medication within 12 hours of treatment', value='22%', regiment='DSUVIA group'), ClinicalOutcome(outcome='percentage of patients taking rescue medication within 12 hours of treatment', value='65%', regiment='placebo')])

In [29]:
outcomes = []
values = []
regiments = []

for result in results:
    for clinical_outcome_list in result:
        for clinical_outcome in clinical_outcome_list[1]:
            print(clinical_outcome)
            outcomes.append(clinical_outcome.outcome)
            values.append(clinical_outcome.value)
            regiments.append(clinical_outcome.regiment)

clinical_outcomes_df = pd.DataFrame().assign(outcome=outcomes,value=values,regiment=regiments)
clinical_outcomes_df

outcome='time-weighted summed pain intensity difference over 12 hours (SPID12)' value='time' regiment='DSUVIA 30 mcg'
outcome='time-weighted summed pain intensity difference over 12 hours (SPID12)' value='placebo' regiment='placebo'
outcome='median time to onset of meaningful pain relief' value='54 minutes' regiment='DSUVIA group'
outcome='median time to onset of meaningful pain relief' value='84 minutes' regiment='placebo'
outcome='percentage of patients taking rescue medication within 12 hours of treatment' value='22%' regiment='DSUVIA group'
outcome='percentage of patients taking rescue medication within 12 hours of treatment' value='65%' regiment='placebo'
outcome='tremor' value='reduced' regiment='ropinirole tablets'
outcome='rigidity' value='eased' regiment='ropinirole tablets'
outcome='bradykinesia' value='improved' regiment='ropinirole tablets'
outcome='postural instability' value='reduced' regiment='ropinirole tablets'
outcome='median overall survival' value='9.0 months' regim

,outcome,value,regiment
0,time-weighted summed pain intensity difference...,time,DSUVIA 30 mcg
1,time-weighted summed pain intensity difference...,placebo,placebo
2,median time to onset of meaningful pain relief,54 minutes,DSUVIA group
3,median time to onset of meaningful pain relief,84 minutes,placebo
4,percentage of patients taking rescue medicatio...,22%,DSUVIA group
...,...,...,...
67,histologic evaluations,effectiveness of mesalamine suppositories was ...,mesalamine suppositories
68,median overall survival,9.0 months,triplet-therapy group
69,median overall survival,5.4 months,control group
70,confirmed response rate,"26% (95 CI, 18 to 35)",triplet-therapy-group


In [30]:
clinical_outcomes_df.to_excel('2024071_llm-orca-test2.xlsx')

In [8]:
results = []
for prompt in tqdm(prompts):
    result = generator(prompt)
    print(prompt)
    print(result)
    results.append(result)

  3%|▎         | 1/31 [7:15:56<217:58:14, 26156.49s/it]

You are a professional medical practicioner with a medical degree. Other doctors \ 
send you clinical trial reports from which you need to extract:

1. The primary outcome metrics from the study
2. The value measured for each primary outcome
3. The therapeutic treatment regiment used to achieve each primary outcome

# EXAMPLE
CLINICAL_TRIAL: The median overall survival was 9.0 months in the triplet-therapy group and 5.4 months in the control group (hazard ratio for death, 0.52; 95% confidence interval [CI], 0.39 to 0.70; P<0.001). The confirmed response rate was 26% (95% CI, 18 to 35) in the triplet-therapy group and 2% (95% CI, 0 to 7) in the control group (P<0.001). The median overall survival in the doublet-therapy group was 8.4 months (hazard ratio for death vs. control, 0.60; 95% CI, 0.45 to 0.79; P<0.001). Adverse events of grade 3 or higher occurred in 58% of patients in the triplet-therapy group, in 50% in the doublet-therapy group, and in 61% in the control group.
RESULTS: {"o

  6%|▋         | 2/31 [10:32:31<142:39:01, 17708.32s/it]

You are a professional medical practicioner with a medical degree. Other doctors \ 
send you clinical trial reports from which you need to extract:

1. The primary outcome metrics from the study
2. The value measured for each primary outcome
3. The therapeutic treatment regiment used to achieve each primary outcome

# EXAMPLE
CLINICAL_TRIAL: The median overall survival was 9.0 months in the triplet-therapy group and 5.4 months in the control group (hazard ratio for death, 0.52; 95% confidence interval [CI], 0.39 to 0.70; P<0.001). The confirmed response rate was 26% (95% CI, 18 to 35) in the triplet-therapy group and 2% (95% CI, 0 to 7) in the control group (P<0.001). The median overall survival in the doublet-therapy group was 8.4 months (hazard ratio for death vs. control, 0.60; 95% CI, 0.45 to 0.79; P<0.001). Adverse events of grade 3 or higher occurred in 58% of patients in the triplet-therapy group, in 50% in the doublet-therapy group, and in 61% in the control group.
RESULTS: {"o

 10%|▉         | 3/31 [18:35:55<177:50:54, 22866.23s/it]

...

You are a professional medical practicioner with a medical degree. Other doctors \ 
send you clinical trial reports from which you need to extract:

1. The primary outcome metrics from the study
2. The value measured for each primary outcome
3. The therapeutic treatment regiment used to achieve each primary outcome

# EXAMPLE
CLINICAL_TRIAL: The median overall survival was 9.0 months in the triplet-therapy group and 5.4 months in the control group (hazard ratio for death, 0.52; 95% confidence interval [CI], 0.39 to 0.70; P<0.001). The confirmed response rate was 26% (95% CI, 18 to 35) in the triplet-therapy group and 2% (95% CI, 0 to 7) in the control group (P<0.001). The median overall survival in the doublet-therapy group was 8.4 months (hazard ratio for death vs. control, 0.60; 95% CI, 0.45 to 0.79; P<0.001). Adverse events of grade 3 or higher occurred in 58% of patients in the triplet-therapy group, in 50% in the doublet-therapy group, and in 61% in the control group.
RESULTS: {"o

 48%|████▊     | 15/31 [84:26:57<72:31:58, 16319.93s/it] 

You are a professional medical practicioner with a medical degree. Other doctors \ 
send you clinical trial reports from which you need to extract:

1. The primary outcome metrics from the study
2. The value measured for each primary outcome
3. The therapeutic treatment regiment used to achieve each primary outcome

# EXAMPLE
CLINICAL_TRIAL: The median overall survival was 9.0 months in the triplet-therapy group and 5.4 months in the control group (hazard ratio for death, 0.52; 95% confidence interval [CI], 0.39 to 0.70; P<0.001). The confirmed response rate was 26% (95% CI, 18 to 35) in the triplet-therapy group and 2% (95% CI, 0 to 7) in the control group (P<0.001). The median overall survival in the doublet-therapy group was 8.4 months (hazard ratio for death vs. control, 0.60; 95% CI, 0.45 to 0.79; P<0.001). Adverse events of grade 3 or higher occurred in 58% of patients in the triplet-therapy group, in 50% in the doublet-therapy group, and in 61% in the control group.
RESULTS: {"o

 48%|████▊     | 15/31 [88:45:09<94:40:10, 21300.63s/it]


KeyboardInterrupt: 